In [0]:
df_filtered = spark.read.parquet("/Volumes/workspace/default/abc/retail_filtered.parquet/")



<h4>Convert InvoiceDate to proper TimestampType.</h4>

In [0]:
from pyspark.sql import functions as F

In [0]:
df_transformed = df_filtered.withColumn(
    "InvoiceDate",
    F.coalesce(
        F.to_timestamp("InvoiceDate", "M/d/yyyy H:mm"),
        F.to_timestamp("InvoiceDate", "MM/dd/yyyy HH:mm"),
        F.to_timestamp("InvoiceDate", "dd/MM/yyyy HH:mm")
    )
)

In [0]:
df_transformed.select("InvoiceDate").show(5, truncate=False)
df_transformed.printSchema()


<h4>Add a new column TotalPrice = Quantity * UnitPrice.</h4>

In [0]:
df_transformed=df_transformed.withColumn('TotalPrice',F.col('Quantity')*F.col('UnitPrice'))
display(df_transformed.limit(10))

<h4>Extract InvoiceYear, InvoiceMonth, and Weekday.</h4>

In [0]:
df_transformed=df_transformed.withColumns({
    'InvoiceYear':F.year('InvoiceDate'),
    'InvoiceMonth':F.month('InvoiceDate'),
    "InvoiceDay": F.dayofmonth("InvoiceDate"),
    "WeekDay": F.date_format("InvoiceDate", "E")
})
display(df_transformed.limit(10))

<h4>Find top 10 product by revenue. </h4>

In [0]:
top_10_products=(df_transformed
.groupby('Description')
.agg(F.sum('TotalPrice').alias('TotalRevenue'))
.orderBy(F.desc('TotalRevenue'))
.limit(10))
